In [1]:
!pip install fcapy
!pip install frozendict
!pip install ipynb
!pip install sparselinear
!pip install bitsets
!pip install bitarray
import torch
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cuda118.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install xgboost
!pip install catboost

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cuda118.html
  Cloning https://github.com/pyg-team/pytorch_geometric.git to c:\users\vasim\appdata\local\temp\pip-req-build-ur_4ud15
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit bd5ae45c74a3fbb6b6ff818476f7651d84313d2a
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git 'C:\Users\vasim\AppData\Local\Temp\pip-req-build-ur_4ud15'


# Import libs

In [2]:
import numpy as np
import pandas as pd
from fcapy.context import FormalContext
from fcapy.lattice import ConceptLattice
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, roc_curve, roc_auc_score
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import sympy

In [3]:
import sys
sys.path.append(r'C:\Users\vasim\Downloads')

In [4]:
import neural_lib as nl

# Prepare Data

In [5]:
df_bin = pd.read_csv(r'C:\Users\vasim\Downloads\TravelInsurancePrediction_binarized_2.csv', index_col=0)
df_bin.columns = [f.split('_')[-1] for f in df_bin.columns]
df_bin.head()

,target,is-age-ng-25,is-age-ng-26,is-age-ng-27,is-age-ng-28,is-age-ng-29,is-age-ng-30,is-age-ng-31,is-age-ng-32,is-age-ng-33,...,is-nof-ng-3,is-nof-ng-4,is-nof-ng-5,is-nof-ng-6,is-nof-ng-7,is-nof-ng-8,is-nof-g-8,is-chronic-disease,is-frequent-flyer,ever-travelled
id,,,,,,,,,,,,,,,,,,,,,
person1,False,False,False,False,False,False,False,True,True,True,...,False,False,False,True,True,True,False,True,False,False
person2,False,False,False,False,False,False,False,True,True,True,...,False,False,False,False,True,True,False,False,False,False
person3,True,False,False,False,False,False,False,False,False,False,...,False,True,True,True,True,True,False,True,False,False
person4,False,False,False,False,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,False,False
person5,False,False,False,False,True,True,True,True,True,True,...,False,False,False,False,False,True,False,True,True,False


In [6]:
df = pd.read_csv(r'C:\Users\vasim\Downloads\TravelInsurancePrediction_numeric.csv', index_col=0)
df.columns = [f.split('_')[-1] for f in df.columns]
df.head()

,Age,Employment Type,GraduateOr0t,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,1,1,400000,6,1,0,0,0
1,31,0,1,1250000,7,0,0,0,0
2,34,0,1,500000,4,1,0,0,1
3,28,0,1,700000,3,1,0,0,0
4,28,0,1,700000,8,1,1,0,0


In [7]:
scaler = preprocessing.MinMaxScaler()
arr_scaled = scaler.fit_transform(df)
df = pd.DataFrame(arr_scaled, columns=df.columns)
df.head()

,Age,Employment Type,GraduateOr0t,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,0.6,1.0,1.0,0.066667,0.571429,1.0,0.0,0.0,0.0
1,0.6,0.0,1.0,0.633333,0.714286,0.0,0.0,0.0,0.0
2,0.9,0.0,1.0,0.133333,0.285714,1.0,0.0,0.0,1.0
3,0.3,0.0,1.0,0.266667,0.142857,1.0,0.0,0.0,0.0
4,0.3,0.0,1.0,0.266667,0.857143,1.0,1.0,0.0,0.0


In [8]:
y_feat_numeric = 'TravelInsurance'
df_test_numeric = df[600:750]
df_train_numeric = df[0:600]

In [9]:
X_train_num, y_train_num = df_train_numeric.drop(y_feat_numeric, axis=1), df_train_numeric[y_feat_numeric]
X_test_num, y_test_num = df_test_numeric.drop(y_feat_numeric, axis=1), df_test_numeric[y_feat_numeric]

In [10]:
y_feat = 'target'
df_train = df_bin[0:600]
df_test = df_bin[600:750]

In [11]:
X_train, y_train = df_train.drop(y_feat, axis=1), df_train[y_feat]
X_test, y_test = df_test.drop(y_feat, axis=1), df_test[y_feat]

# Build Monotone Concept Lattice

In [12]:
K_train = FormalContext.from_pandas(X_train)
K_train

FormalContext (600 objects, 29 attributes, 8318 connections)
         |is-age-ng-25|is-age-ng-26|is-age-ng-27|is-age-ng-28|is-age-ng-29|...|is-nof-ng-8|is-nof-g-8|is-chronic-disease|is-frequent-flyer|ever-travelled|
person1  |            |            |            |            |            |...|          X|          |                 X|                 |              |
person2  |            |            |            |            |            |...|          X|          |                  |                 |              |
person3  |            |            |            |            |            |...|          X|          |                 X|                 |              |
person4  |            |            |            |           X|           X|...|          X|          |                 X|                 |              |
person5  |            |            |            |           X|           X|...|          X|          |                 X|                X|              |
person6  

In [13]:
L = ConceptLattice.from_context(K_train, algo='Sofia', is_monotone=True)
len(L)

97

In [14]:
for c in L:
    y_preds = np.zeros(K_train.n_objects)
    y_preds[list(c.extent_i)] = 1
    c.measures['accuracy'] = accuracy_score(y_train, y_preds)

In [15]:
best_concepts = list(L.measures['accuracy'].argsort()[::-1][:80])

assert len({g_i for c in L[best_concepts] for g_i in c.extent_i})==K_train.n_objects, "Selected concepts do not cover all train objects"

In [16]:
cn = nl.ConceptNetwork.from_lattice(L, best_concepts, sorted(set(y_train)))

In [17]:
cn.fit(X_train, y_train)

In [18]:
y_pred = cn.predict(X_test).numpy()
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score on test data: ", accuracy)

Accuracy Score on test data:  0.66


# Decision Tree

In [19]:
cn_DT = DecisionTreeClassifier()

In [20]:
param_grid_DT = {'max_features': ['sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [3, 4, 5, 6, 7, 8, 9, 10, 11],
              'criterion' :['gini', 'entropy']
             }

In [21]:
grid_search_DT_as = GridSearchCV(estimator=cn_DT, param_grid=param_grid_DT, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_DT_as.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_DT_as.best_params_)
print("Best cross-validation score: ", grid_search_DT_as.best_score_)

Best parameters found:  {'ccp_alpha': 0.01, 'criterion': 'entropy', 'max_depth': 11, 'max_features': 'log2'}
Best cross-validation score:  0.8116666666666668


In [22]:
grid_search_DT_ps = GridSearchCV(estimator=cn_DT, param_grid=param_grid_DT, cv=5, scoring='precision', n_jobs=-1)
grid_search_DT_ps.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_DT_ps.best_params_)
print("Best cross-validation score: ", grid_search_DT_ps.best_score_)

Best parameters found:  {'ccp_alpha': 0.01, 'criterion': 'gini', 'max_depth': 11, 'max_features': 'log2'}
Best cross-validation score:  0.9046494355317884


In [23]:
grid_search_DT_rs = GridSearchCV(estimator=cn_DT, param_grid=param_grid_DT, cv=5, scoring='recall', n_jobs=-1)
grid_search_DT_rs.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_DT_rs.best_params_)
print("Best cross-validation score: ", grid_search_DT_rs.best_score_)

Best parameters found:  {'ccp_alpha': 0.001, 'criterion': 'gini', 'max_depth': 9, 'max_features': 'log2'}
Best cross-validation score:  0.5842044134727062


In [24]:
grid_search_DT_f1s = GridSearchCV(estimator=cn_DT, param_grid=param_grid_DT, cv=5, scoring='f1', n_jobs=-1)
grid_search_DT_f1s.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_DT_f1s.best_params_)
print("Best cross-validation score: ", grid_search_DT_f1s.best_score_)

Best parameters found:  {'ccp_alpha': 0.01, 'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2'}
Best cross-validation score:  0.680813751194972


In [25]:
grid_search_DT_ras = GridSearchCV(estimator=cn_DT, param_grid=param_grid_DT, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
grid_search_DT_ras.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_DT_ras.best_params_)
print("Best cross-validation score: ", grid_search_DT_ras.best_score_)

Best parameters found:  {'ccp_alpha': 0.01, 'criterion': 'entropy', 'max_depth': 4, 'max_features': 'log2'}
Best cross-validation score:  0.7753567913339448


In [26]:
y_DT_pred_as = grid_search_DT_as.predict(X_test_num)
y_DT_pred_ps = grid_search_DT_ps.predict(X_test_num)
y_DT_pred_rs = grid_search_DT_rs.predict(X_test_num)
y_DT_pred_f1s = grid_search_DT_f1s.predict(X_test_num)
y_DT_pred_ras = grid_search_DT_ras.predict(X_test_num)

In [27]:
accuracy_DT = accuracy_score(y_test_num, y_DT_pred_as)
print("Accuracy Score on test data: ", accuracy_DT)
precision_DT = precision_score(y_test_num, y_DT_pred_ps)
print("Precision Score on test data: ", precision_DT)
recall_DT = recall_score(y_test_num, y_DT_pred_rs)
print("Recall Score on test data: ", recall_DT)
f1_DT = f1_score(y_test_num, y_DT_pred_f1s)
print("F1 Score on test data: ", f1_DT)
roc_auc_DT = roc_auc_score(y_test_num, y_DT_pred_ras)
print("ROC AUC Score on test data: ", roc_auc_DT)

Accuracy Score on test data:  0.84
Precision Score on test data:  0.75
Recall Score on test data:  0.7058823529411765
F1 Score on test data:  0.7415730337078652
ROC AUC Score on test data:  0.6746880570409983


# Random Forest

In [28]:
clf_RF = RandomForestClassifier(class_weight="balanced")

In [29]:
param_grid_RF = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'criterion': ['gini', 'entropy', 'log_loss']
}

In [30]:
grid_search_RF_as = GridSearchCV(estimator=clf_RF, param_grid=param_grid_RF, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_RF_as.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_RF_as.best_params_)
print("Best cross-validation score: ", grid_search_RF_as.best_score_)

C:\Users\vasim\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found:  {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'log2', 'n_estimators': 200}
Best cross-validation score:  0.8316666666666667


In [31]:
grid_search_RF_ps = GridSearchCV(estimator=clf_RF, param_grid=param_grid_RF, cv=3, scoring='precision', n_jobs=-1)
grid_search_RF_ps.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_RF_ps.best_params_)
print("Best cross-validation score: ", grid_search_RF_ps.best_score_)

Best parameters found:  {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 100}
Best cross-validation score:  0.8851851851851853


In [32]:
grid_search_RF_rs = GridSearchCV(estimator=clf_RF, param_grid=param_grid_RF, cv=3, scoring='recall', n_jobs=-1)
grid_search_RF_rs.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_RF_rs.best_params_)
print("Best cross-validation score: ", grid_search_RF_rs.best_score_)

Best parameters found:  {'criterion': 'entropy', 'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 400}
Best cross-validation score:  0.6135265700483092


In [33]:
grid_search_RF_f1s = GridSearchCV(estimator=clf_RF, param_grid=param_grid_RF, cv=3, scoring='f1', n_jobs=-1)
grid_search_RF_f1s.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_RF_f1s.best_params_)
print("Best cross-validation score: ", grid_search_RF_f1s.best_score_)

Best parameters found:  {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'log2', 'n_estimators': 500}
Best cross-validation score:  0.7070332660015518


In [34]:
grid_search_RF_ras = GridSearchCV(estimator=clf_RF, param_grid=param_grid_RF, cv=3, scoring='roc_auc_ovr', n_jobs=-1)
grid_search_RF_ras.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_RF_ras.best_params_)
print("Best cross-validation score: ", grid_search_RF_ras.best_score_)

Best parameters found:  {'criterion': 'log_loss', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 100}
Best cross-validation score:  0.7853929269461961


In [35]:
y_RF_pred_as = grid_search_RF_as.predict(X_test_num)
y_RF_pred_ps = grid_search_RF_ps.predict(X_test_num)
y_RF_pred_rs = grid_search_RF_rs.predict(X_test_num)
y_RF_pred_f1s = grid_search_RF_f1s.predict(X_test_num)
y_RF_pred_ras = grid_search_RF_ras.predict(X_test_num)

In [36]:
accuracy_RF = accuracy_score(y_test_num, y_RF_pred_as)
print("Accuracy Score on test data: ", accuracy_RF)
precision_RF = precision_score(y_test_num, y_RF_pred_ps)
print("Precision Score on test data: ", precision_RF)
recall_RF = recall_score(y_test_num, y_RF_pred_rs)
print("Recall Score on test data: ", recall_RF)
f1_RF = f1_score(y_test_num, y_RF_pred_f1s)
print("F1 Score on test data: ", f1_DT)
roc_auc_RF = roc_auc_score(y_test_num, y_RF_pred_ras)
print("ROC AUC Score on test data: ", roc_auc_RF)

Accuracy Score on test data:  0.8666666666666667
Precision Score on test data:  0.8604651162790697
Recall Score on test data:  0.7450980392156863
F1 Score on test data:  0.7415730337078652
ROC AUC Score on test data:  0.8475935828877006


# XGBoost

In [37]:
clf_XG = xgb.XGBClassifier()

In [38]:
param_grid_XG = {'nthread':[10],
                 'colsample_bytree' : [0.5, 0.6, 0.7, 0.8, 0.9], 
                 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
                 'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
                 'n_estimators': [100, 200, 300, 400, 500, 600]}

In [39]:
grid_search_XG_as = GridSearchCV(estimator=clf_XG, param_grid=param_grid_XG, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_XG_as.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_XG_as.best_params_)
print("Best cross-validation score: ", grid_search_XG_as.best_score_)

Best parameters found:  {'colsample_bytree': 0.5, 'max_depth': 3, 'n_estimators': 100, 'nthread': 10, 'subsample': 0.9}
Best cross-validation score:  0.8033333333333333


In [40]:
grid_search_XG_ps = GridSearchCV(estimator=clf_XG, param_grid=param_grid_XG, cv=3, scoring='precision', n_jobs=-1)
grid_search_XG_ps.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_XG_ps.best_params_)
print("Best cross-validation score: ", grid_search_XG_ps.best_score_)

Best parameters found:  {'colsample_bytree': 0.5, 'max_depth': 3, 'n_estimators': 100, 'nthread': 10, 'subsample': 0.9}
Best cross-validation score:  0.7758251448727639


In [41]:
grid_search_XG_rs = GridSearchCV(estimator=clf_XG, param_grid=param_grid_XG, cv=3, scoring='recall', n_jobs=-1)
grid_search_XG_rs.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_XG_rs.best_params_)
print("Best cross-validation score: ", grid_search_XG_rs.best_score_)

Best parameters found:  {'colsample_bytree': 0.7, 'max_depth': 9, 'n_estimators': 100, 'nthread': 10, 'subsample': 0.7}
Best cross-validation score:  0.6425120772946861


In [42]:
grid_search_XG_f1s = GridSearchCV(estimator=clf_XG, param_grid=param_grid_XG, cv=3, scoring='f1', n_jobs=-1)
grid_search_XG_f1s.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_XG_f1s.best_params_)
print("Best cross-validation score: ", grid_search_XG_f1s.best_score_)

Best parameters found:  {'colsample_bytree': 0.5, 'max_depth': 3, 'n_estimators': 100, 'nthread': 10, 'subsample': 0.9}
Best cross-validation score:  0.6809987256356599


In [43]:
grid_search_XG_ras = GridSearchCV(estimator=clf_XG, param_grid=param_grid_XG, cv=3, scoring='roc_auc_ovr', n_jobs=-1)
grid_search_XG_ras.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_XG_ras.best_params_)
print("Best cross-validation score: ", grid_search_XG_ras.best_score_)

Best parameters found:  {'colsample_bytree': 0.8, 'max_depth': 5, 'n_estimators': 100, 'nthread': 10, 'subsample': 0.6}
Best cross-validation score:  0.7717667883615444


In [44]:
y_XG_pred_as = grid_search_XG_as.predict(X_test_num)
y_XG_pred_ps = grid_search_XG_ps.predict(X_test_num)
y_XG_pred_rs = grid_search_XG_rs.predict(X_test_num)
y_XG_pred_f1s = grid_search_XG_f1s.predict(X_test_num)
y_XG_pred_ras = grid_search_XG_ras.predict(X_test_num)

In [45]:
accuracy_XG = accuracy_score(y_test_num, y_XG_pred_as)
print("Accuracy Score on test data: ", accuracy_XG)
precision_XG = precision_score(y_test_num, y_XG_pred_ps)
print("Precision Score on test data: ", precision_XG)
recall_XG = recall_score(y_test_num, y_XG_pred_rs)
print("Recall Score on test data: ", recall_XG)
f1_XG = f1_score(y_test_num, y_XG_pred_f1s)
print("F1 Score on test data: ", f1_DT)
roc_auc_XG = roc_auc_score(y_test_num, y_XG_pred_ras)
print("ROC AUC Score on test data: ", roc_auc_XG)

Accuracy Score on test data:  0.8533333333333334
Precision Score on test data:  0.8536585365853658
Recall Score on test data:  0.7254901960784313
F1 Score on test data:  0.7415730337078652
ROC AUC Score on test data:  0.812537136066548


# CatBoost

In [46]:
clf_CB=CatBoostClassifier(loss_function='Logloss', random_state=42)

In [47]:
param_grid_CB = {
    'iterations': [100, 200, 300, 400, 500, 600],
    'learning_rate': [0.2, 0.1, 0.05, 0.01],
    'depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

In [48]:
grid_search_CB_as = GridSearchCV(estimator=clf_CB, param_grid=param_grid_CB, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_CB_as.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_CB_as.best_params_)
print("Best cross-validation score: ", grid_search_CB_as.best_score_)

0:	learn: 0.6276470	total: 53.7ms	remaining: 5.31s
1:	learn: 0.5847199	total: 54ms	remaining: 2.65s
2:	learn: 0.5532408	total: 54.2ms	remaining: 1.75s
3:	learn: 0.5333715	total: 54.4ms	remaining: 1.31s
4:	learn: 0.5192353	total: 54.6ms	remaining: 1.04s
5:	learn: 0.5077749	total: 54.8ms	remaining: 859ms
6:	learn: 0.5022489	total: 55ms	remaining: 731ms
7:	learn: 0.4958642	total: 55.2ms	remaining: 635ms
8:	learn: 0.4878928	total: 55.4ms	remaining: 560ms
9:	learn: 0.4843846	total: 55.6ms	remaining: 500ms
10:	learn: 0.4802313	total: 55.7ms	remaining: 451ms
11:	learn: 0.4791065	total: 55.9ms	remaining: 410ms
12:	learn: 0.4768860	total: 56.1ms	remaining: 375ms
13:	learn: 0.4703731	total: 56.2ms	remaining: 346ms
14:	learn: 0.4681357	total: 56.4ms	remaining: 320ms
15:	learn: 0.4659176	total: 56.6ms	remaining: 297ms
16:	learn: 0.4630902	total: 56.8ms	remaining: 277ms
17:	learn: 0.4598043	total: 57ms	remaining: 259ms
18:	learn: 0.4582510	total: 57.1ms	remaining: 244ms
19:	learn: 0.4571182	total: 

In [49]:
grid_search_CB_ps = GridSearchCV(estimator=clf_CB, param_grid=param_grid_CB, cv=3, scoring='precision', n_jobs=-1)
grid_search_CB_ps.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_CB_ps.best_params_)
print("Best cross-validation score: ", grid_search_CB_ps.best_score_)

0:	learn: 0.6889666	total: 434us	remaining: 86.5ms
1:	learn: 0.6855075	total: 1.11ms	remaining: 110ms
2:	learn: 0.6814872	total: 1.61ms	remaining: 106ms
3:	learn: 0.6775547	total: 2.09ms	remaining: 102ms
4:	learn: 0.6737052	total: 2.56ms	remaining: 99.8ms
5:	learn: 0.6699466	total: 3.04ms	remaining: 98.2ms
6:	learn: 0.6662675	total: 3.5ms	remaining: 96.4ms
7:	learn: 0.6626690	total: 3.99ms	remaining: 95.8ms
8:	learn: 0.6591503	total: 4.46ms	remaining: 94.8ms
9:	learn: 0.6569652	total: 4.92ms	remaining: 93.4ms
10:	learn: 0.6535730	total: 5.41ms	remaining: 93ms
11:	learn: 0.6507490	total: 5.85ms	remaining: 91.7ms
12:	learn: 0.6474867	total: 6.33ms	remaining: 91.1ms
13:	learn: 0.6443005	total: 6.8ms	remaining: 90.4ms
14:	learn: 0.6417884	total: 7.27ms	remaining: 89.7ms
15:	learn: 0.6391842	total: 7.73ms	remaining: 88.9ms
16:	learn: 0.6372934	total: 8.18ms	remaining: 88ms
17:	learn: 0.6343318	total: 8.63ms	remaining: 87.3ms
18:	learn: 0.6325184	total: 9.07ms	remaining: 86.4ms
19:	learn: 0.

In [50]:
grid_search_CB_rs = GridSearchCV(estimator=clf_CB, param_grid=param_grid_CB, cv=3, scoring='recall', n_jobs=-1)
grid_search_CB_rs.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_CB_rs.best_params_)
print("Best cross-validation score: ", grid_search_CB_rs.best_score_)

0:	learn: 0.6080944	total: 523us	remaining: 261ms
1:	learn: 0.5635495	total: 1.19ms	remaining: 296ms
2:	learn: 0.5313326	total: 1.68ms	remaining: 278ms
3:	learn: 0.5126376	total: 2.19ms	remaining: 272ms
4:	learn: 0.5046608	total: 2.7ms	remaining: 267ms
5:	learn: 0.4956921	total: 3.21ms	remaining: 264ms
6:	learn: 0.4862675	total: 3.7ms	remaining: 261ms
7:	learn: 0.4771353	total: 4.22ms	remaining: 259ms
8:	learn: 0.4679050	total: 4.73ms	remaining: 258ms
9:	learn: 0.4632111	total: 5.25ms	remaining: 257ms
10:	learn: 0.4606159	total: 5.76ms	remaining: 256ms
11:	learn: 0.4594676	total: 6.28ms	remaining: 255ms
12:	learn: 0.4582547	total: 6.79ms	remaining: 255ms
13:	learn: 0.4570740	total: 7.31ms	remaining: 254ms
14:	learn: 0.4557564	total: 7.83ms	remaining: 253ms
15:	learn: 0.4533308	total: 8.31ms	remaining: 252ms
16:	learn: 0.4516063	total: 8.83ms	remaining: 251ms
17:	learn: 0.4509584	total: 9.34ms	remaining: 250ms
18:	learn: 0.4447666	total: 9.86ms	remaining: 249ms
19:	learn: 0.4442269	tota

In [51]:
grid_search_CB_f1s = GridSearchCV(estimator=clf_CB, param_grid=param_grid_CB, cv=3, scoring='f1', n_jobs=-1)
grid_search_CB_f1s.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_CB_f1s.best_params_)
print("Best cross-validation score: ", grid_search_CB_f1s.best_score_)

0:	learn: 0.6276470	total: 240us	remaining: 23.8ms
1:	learn: 0.5847199	total: 496us	remaining: 24.4ms
2:	learn: 0.5532408	total: 675us	remaining: 21.8ms
3:	learn: 0.5333715	total: 845us	remaining: 20.3ms
4:	learn: 0.5192353	total: 1.04ms	remaining: 19.8ms
5:	learn: 0.5077749	total: 1.22ms	remaining: 19ms
6:	learn: 0.5022489	total: 1.38ms	remaining: 18.3ms
7:	learn: 0.4958642	total: 1.55ms	remaining: 17.9ms
8:	learn: 0.4878928	total: 1.72ms	remaining: 17.4ms
9:	learn: 0.4843846	total: 1.89ms	remaining: 17ms
10:	learn: 0.4802313	total: 2.06ms	remaining: 16.7ms
11:	learn: 0.4791065	total: 2.22ms	remaining: 16.3ms
12:	learn: 0.4768860	total: 2.39ms	remaining: 16ms
13:	learn: 0.4703731	total: 2.55ms	remaining: 15.7ms
14:	learn: 0.4681357	total: 2.71ms	remaining: 15.4ms
15:	learn: 0.4659176	total: 2.88ms	remaining: 15.1ms
16:	learn: 0.4630902	total: 3.05ms	remaining: 14.9ms
17:	learn: 0.4598043	total: 3.22ms	remaining: 14.7ms
18:	learn: 0.4582510	total: 3.39ms	remaining: 14.4ms
19:	learn: 0.

In [52]:
grid_search_CB_ras = GridSearchCV(estimator=clf_CB, param_grid=param_grid_CB, cv=3, scoring='roc_auc_ovr', n_jobs=-1)
grid_search_CB_ras.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_CB_ras.best_params_)
print("Best cross-validation score: ", grid_search_CB_ras.best_score_)

0:	learn: 0.6894531	total: 391us	remaining: 38.8ms
1:	learn: 0.6862624	total: 822us	remaining: 40.3ms
2:	learn: 0.6832624	total: 1.17ms	remaining: 37.9ms
3:	learn: 0.6800204	total: 1.48ms	remaining: 35.5ms
4:	learn: 0.6770671	total: 1.78ms	remaining: 33.9ms
5:	learn: 0.6740915	total: 2.09ms	remaining: 32.7ms
6:	learn: 0.6714896	total: 2.41ms	remaining: 32ms
7:	learn: 0.6687351	total: 2.71ms	remaining: 31.2ms
8:	learn: 0.6657163	total: 3.04ms	remaining: 30.8ms
9:	learn: 0.6630885	total: 3.34ms	remaining: 30.1ms
10:	learn: 0.6608303	total: 3.65ms	remaining: 29.6ms
11:	learn: 0.6578313	total: 3.96ms	remaining: 29ms
12:	learn: 0.6551627	total: 4.26ms	remaining: 28.5ms
13:	learn: 0.6522997	total: 4.55ms	remaining: 28ms
14:	learn: 0.6498259	total: 4.86ms	remaining: 27.5ms
15:	learn: 0.6473324	total: 5.18ms	remaining: 27.2ms
16:	learn: 0.6448181	total: 5.5ms	remaining: 26.8ms
17:	learn: 0.6424372	total: 5.81ms	remaining: 26.5ms
18:	learn: 0.6403244	total: 6.12ms	remaining: 26.1ms
19:	learn: 0

In [53]:
y_CB_pred_as = grid_search_CB_as.predict(X_test_num)
y_CB_pred_ps = grid_search_CB_ps.predict(X_test_num)
y_CB_pred_rs = grid_search_CB_rs.predict(X_test_num)
y_CB_pred_f1s = grid_search_CB_f1s.predict(X_test_num)
y_CB_pred_ras = grid_search_CB_ras.predict(X_test_num)

In [54]:
accuracy_CB = accuracy_score(y_test_num, y_CB_pred_as)
print("Accuracy Score on test data: ", accuracy_CB)
precision_CB = precision_score(y_test_num, y_CB_pred_ps)
print("Precision Score on test data: ", precision_CB)
recall_CB = recall_score(y_test_num, y_CB_pred_rs)
print("Recall Score on test data: ", recall_CB)
f1_CB = f1_score(y_test_num, y_CB_pred_f1s)
print("F1 Score on test data: ", f1_DT)
roc_auc_CB = roc_auc_score(y_test_num, y_CB_pred_ras)
print("ROC AUC Score on test data: ", roc_auc_CB)

Accuracy Score on test data:  0.8666666666666667
Precision Score on test data:  0.88
Recall Score on test data:  0.6862745098039216
F1 Score on test data:  0.7415730337078652
ROC AUC Score on test data:  0.7935234699940582


# KNN

In [55]:
clf_KNN = KNeighborsClassifier()

In [56]:
param_grid_KNN = {
    'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'leaf_size': [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
    'p': [1, 2],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'chebyshev']}

In [57]:
grid_search_KNN_as = GridSearchCV(estimator=clf_KNN, param_grid=param_grid_KNN, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_KNN_as.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_KNN_as.best_params_)
print("Best cross-validation score: ", grid_search_KNN_as.best_score_)

Best parameters found:  {'leaf_size': 1, 'metric': 'minkowski', 'n_neighbors': 6, 'p': 1, 'weights': 'uniform'}
Best cross-validation score:  0.7766666666666667


In [58]:
grid_search_KNN_ps = GridSearchCV(estimator=clf_KNN, param_grid=param_grid_KNN, cv=3, scoring='precision', n_jobs=-1)
grid_search_KNN_ps.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_KNN_ps.best_params_)
print("Best cross-validation score: ", grid_search_KNN_ps.best_score_)

Best parameters found:  {'leaf_size': 25, 'metric': 'minkowski', 'n_neighbors': 4, 'p': 2, 'weights': 'uniform'}
Best cross-validation score:  0.8292448292448292


In [59]:
grid_search_KNN_rs = GridSearchCV(estimator=clf_KNN, param_grid=param_grid_KNN, cv=3, scoring='recall', n_jobs=-1)
grid_search_KNN_rs.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_KNN_rs.best_params_)
print("Best cross-validation score: ", grid_search_KNN_rs.best_score_)

Best parameters found:  {'leaf_size': 50, 'metric': 'minkowski', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Best cross-validation score:  0.5700483091787439


C:\Users\vasim\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [60]:
grid_search_KNN_f1s = GridSearchCV(estimator=clf_KNN, param_grid=param_grid_KNN, cv=3, scoring='f1', n_jobs=-1)
grid_search_KNN_f1s.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_KNN_f1s.best_params_)
print("Best cross-validation score: ", grid_search_KNN_f1s.best_score_)

Best parameters found:  {'leaf_size': 1, 'metric': 'minkowski', 'n_neighbors': 11, 'p': 1, 'weights': 'distance'}
Best cross-validation score:  0.6051215721662406


In [61]:
grid_search_KNN_ras = GridSearchCV(estimator=clf_KNN, param_grid=param_grid_KNN, cv=3, scoring='roc_auc_ovr', n_jobs=-1)
grid_search_KNN_ras.fit(X_train_num, y_train_num)
print("Best parameters found: ", grid_search_KNN_ras.best_params_)
print("Best cross-validation score: ", grid_search_KNN_ras.best_score_)

Best parameters found:  {'leaf_size': 25, 'metric': 'minkowski', 'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}
Best cross-validation score:  0.7495666924807317


In [62]:
y_KNN_pred_as = grid_search_KNN_as.predict(X_test_num)
y_KNN_pred_ps = grid_search_KNN_ps.predict(X_test_num)
y_KNN_pred_rs = grid_search_KNN_rs.predict(X_test_num)
y_KNN_pred_f1s = grid_search_KNN_f1s.predict(X_test_num)
y_KNN_pred_ras = grid_search_KNN_ras.predict(X_test_num)

In [63]:
accuracy_KNN = accuracy_score(y_test_num, y_KNN_pred_as)
print("Accuracy Score on test data: ", accuracy_KNN)
precision_KNN = precision_score(y_test_num, y_KNN_pred_ps)
print("Precision Score on test data: ", precision_KNN)
recall_KNN = recall_score(y_test_num, y_KNN_pred_rs)
print("Recall Score on test data: ", recall_KNN)
f1_KNNB = f1_score(y_test_num, y_KNN_pred_f1s)
print("F1 Score on test data: ", f1_DT)
roc_auc_KNN = roc_auc_score(y_test_num, y_KNN_pred_ras)
print("ROC AUC Score on test data: ", roc_auc_KNN)

Accuracy Score on test data:  0.8066666666666666
Precision Score on test data:  0.8064516129032258
Recall Score on test data:  0.6470588235294118
F1 Score on test data:  0.7415730337078652
ROC AUC Score on test data:  0.7540106951871658
